In [13]:
import random
import pandas as pd

In [14]:
# seed
random.seed(50)

# number of customers
size = 100

# number of servers
m = 30

# number of selected servers
d = 10

# series of customer
cust_id = [i for i in range(size)]

# exponential arrival rate (lambda)
lamb = 1/3

# generate mu (service time rate)
mu = 1/2

# inter arrival time
inter_arrival_time = [random.expovariate(lamb) for i in range(1, size+1)]

# service time
service_time = [random.expovariate(mu) for i in range(1, size+1)]

# arrival time
arrival = [0]*(size)
arrival[0] = inter_arrival_time[0]

In [15]:
# calculate arrival time
for i in range(1, size):
    arrival[i] = inter_arrival_time[i] + arrival[i-1]
    
# time service begin
tsb = [0]*size

# server waiting in queue
twq = [0]*size

# time service ends
tse = [0]*size

# time server spent in the system
tss = [0]*size

# system idle
idle = [0]*size


tsb[0] = arrival[0]
tse[0] = tsb[0] + service_time[0]
tss[0] = service_time[0]

In [16]:
for i in range(1, size):
    # Time Service Begin 
    tsb[i] = max(arrival[i], tse[i-1])
    
    # Time customer waiting in queue
    twq[i] = tsb[i]-arrival[i]
    
    # Time service ends
    tse[i] = tsb[i] + service_time[i]  
    
    # Time server Spend in the system
    tss[i] = tse[i] - arrival[i]
    
    # Time when system remains idle
    if (arrival[i]>tse[i-1]):
        idle[i] = arrival[i]-tse[i-1]
    else:
        idle[i] = 0 

In [17]:
# select d servers from m servers
server_table = {}
count = 0

while count < d:
    value = random.randrange(0, m)
    if value not in server_table:
        count += 1
        server_table[value] = 0
    
server_table

{22: 0, 27: 0, 17: 0, 16: 0, 3: 0, 25: 0, 23: 0, 7: 0, 21: 0, 4: 0}

In [18]:
# check queue length
def check_queue_length(server_table):
    res = [2**31-1, 2**31-1]
    for k in server_table:
        if server_table[k] < res[1]:
            res = [k, server_table[k]]
    return res

In [19]:
# randmin algorithm
server_id = []

for i in range(size):
    # pick the queue with the shortest length
    queue = check_queue_length(server_table)
    
    # update the selected queue length
    key_id = queue[0] 
    server_table[key_id] += 1
    
    # add server id into the server_id list
    server_id.append(key_id)

In [20]:
table = {'Customer id': cust_id, 'arrival time': arrival, 'service begins': tsb, 'service time': service_time, 
         'service end': tse, 'time in queue': twq, 'time in system': tss, 'system idle': idle, 'server id': server_id}

df = pd.DataFrame(table)
df

,Customer id,arrival time,service begins,service time,service end,time in queue,time in system,system idle,server id
0,0,2.064697,2.064697,0.308739,2.373437,0.000000,0.308739,0.000000,22
1,1,2.993146,2.993146,0.744973,3.738119,0.000000,0.744973,0.619709,27
2,2,6.036603,6.036603,2.676695,8.713298,0.000000,2.676695,2.298484,17
3,3,6.869718,8.713298,6.359395,15.072693,1.843580,8.202974,0.000000,16
4,4,8.792342,15.072693,0.031905,15.104598,6.280350,6.312255,0.000000,3
...,...,...,...,...,...,...,...,...,...
95,95,297.144541,297.144541,1.101023,298.245564,0.000000,1.101023,3.404180,25
96,96,297.501895,298.245564,0.249814,298.495377,0.743669,0.993483,0.000000,23
97,97,297.777945,298.495377,1.310756,299.806133,0.717432,2.028188,0.000000,7
98,98,298.299753,299.806133,0.565752,300.371886,1.506381,2.072133,0.000000,21


In [21]:
q_len = []
for key in server_table:
    q_len.append(server_table[key])

q_len

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [22]:
# number of customers
print(f'Number of customers in the servers: {size}')

# max workload
print(f'longest queue length amongst the m={m} servers: {max(q_len)}')

# average workload
print(f'average queue length amongst the m={m} servers {sum(q_len)/size:2f}')

# average time in the system
avg_sys = sum(tss)/len(tss)

print(f'average time in the system per customer {avg_sys:6f}')

Number of customers in the servers: 100
longest queue length amongst the m=30 servers: 10
average queue length amongst the m=30 servers 1.000000
average time in the system per customer 3.868261
